In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
import Scan
soc = TopSoc()

In [ ]:
fMixer = soc.fsIn/2/2
soc.set_mixer(fMixer)

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
outCh = 123
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, doPlot=True)


In [ ]:
print(nominalDelay)

In [ ]:
dfs = scan.mndScan['dfs']
xs = scan.mndScan['xs']
plt.plot(dfs,np.real(xs)/np.abs(xs), '.-')
plt.plot(dfs,np.imag(xs)/np.abs(xs), '.-')


In [ ]:
from scipy.optimize import minimize

def fun(pars, xs, dfs):
    phiFits = pars[0] + pars[1]*dfs
    phiData = np.angle(xs)
    dPhis = np.angle(np.exp(1j*(phiFits-phiData)))
    retval = np.power(dPhis,2).mean()
    return retval

phi0 = np.angle(xs[len(xs)//2,0])
guess = np.array([phi0, 0.0])
rv = minimize(fun, guess, args=(xs[:,0],dfs))
if rv.success:
    phi0 = rv.x[0]
    delay = rv.x[1]
    dfFits = np.linspace(dfs.min(),dfs.max(),100)
    xFits = np.exp(1j*(phi0 + delay*dfFits))
    plt.plot(dfs,np.real(xs[:,0])/np.abs(xs[:,0]),'b.', label="data I")
    plt.plot(dfFits, np.real(xFits), 'b', alpha=0.4, label="fit I")
    plt.plot(dfs,np.imag(xs[:,0])/np.abs(xs[:,0]),'r.', label="data Q")
    plt.plot(dfFits, np.imag(xFits), 'r', alpha=0.4, label="fit Q")
    plt.legend()


In [ ]:
rv.success